In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load and preprocess the data
file_path = '/content/NIFTY 50 - Minute data.csv'  # Adjust this path if necessary
data = pd.read_csv(file_path)

data.rename(columns={
    'date': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close'
}, inplace=True)

columns_to_keep = ['Date', 'Close']
data = data[columns_to_keep]

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Prepare the dataset for LSTM
def create_dataset(dataset, time_step=1):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        X.append(a)
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60  # Number of minutes to look back
dataset = data['Close'].values
dataset = dataset.reshape(-1, 1)

X, y = create_dataset(dataset, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data into training and testing sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot the results
plt.figure(figsize=(14, 8))
plt.plot(data.index[-len(y_test):], y_test, label='True Price', color='b')
plt.plot(data.index[-len(predictions):], predictions, label='Predicted Price', color='r')
plt.title('LSTM Stock Price Prediction')
plt.legend()
plt.show()

# Generate trading signals
data['Predicted_Close'] = np.nan
data.iloc[-len(predictions):, data.columns.get_loc('Predicted_Close')] = predictions.flatten()

data['Signal'] = 0.0
data['Signal'] = np.where(data['Predicted_Close'] > data['Close'], 1.0, 0.0)  # Buy signal
data['Signal'] = np.where(data['Predicted_Close'] < data['Close'], -1.0, data['Signal'])  # Sell signal
data['Position'] = data['Signal'].diff()  # 1.0 = Buy, -1.0 = Sell

# Plot the close price and signals
plt.figure(figsize=(14, 8))

plt.plot(data['Close'], label='Close Price', alpha=0.5)
plt.plot(data['Predicted_Close'], label='Predicted Price', alpha=0.75)

plt.plot(data[data['Position'] == 1.0].index,
         data['Close'][data['Position'] == 1.0],
         '^', markersize=10, color='g', lw=0, label='Buy Signal')

plt.plot(data[data['Position'] == -1.0].index,
         data['Close'][data['Position'] == -1.0],
         'v', markersize=10, color='r', lw=0, label='Sell Signal')

plt.title('Trading Strategy Using LSTM Predictions')
plt.legend()
plt.show()

# Backtesting
data['Strategy_Returns'] = data['Close'].pct_change() * data['Signal'].shift(1)
data['Cumulative_Strategy_Returns'] = (data['Strategy_Returns'] + 1).cumprod()
data['Cumulative_Market_Returns'] = (data['Close'].pct_change() + 1).cumprod()

plt.figure(figsize=(14, 8))
plt.plot(data['Cumulative_Strategy_Returns'], label='Strategy Returns', color='g')
plt.plot(data['Cumulative_Market_Returns'], label='Market Returns', color='b')
plt.title('Strategy vs Market Performance')
plt.legend()
plt.show()

strategy_total_return = data['Cumulative_Strategy_Returns'].iloc[-1]
market_total_return = data['Cumulative_Market_Returns'].iloc[-1]

print(f"Strategy Total Return: {strategy_total_return:.2f}")
print(f"Market Total Return: {market_total_return:.2f}")
